In [8]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import pickle
import json
import matplotlib.pyplot as plt
from glob import glob
import time
import copy
from tqdm import tqdm
from transformers import BertLMHeadModel, BartTokenizer, BartForConditionalGeneration, BartConfig, BartForSequenceClassification, BertTokenizer, BertConfig, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification, PegasusForConditionalGeneration, PegasusTokenizer, T5Tokenizer, T5ForConditionalGeneration, BertGenerationEncoder, BertGenerationDecoder, EncoderDecoderConfig, EncoderDecoderModel
from data import ZuCo_dataset
from model_decoding import BrainTranslator, BrainTranslatorNaive
from config import get_config


In [7]:
import pandas as pd

# 파일 경로
file_path = 'score_results/task1_task2_task3-BrainTranslator.txt'


# 파일 내용을 한 줄씩 읽어와서 리스트로 변환
with open(file_path, 'r') as file:
    lines = file.readlines()

# 각 줄을 '=' 기준으로 나누어 튜플로 변환 후 리스트에 저장
data_tuples = [tuple(map(str.strip, line.split('='))) for line in lines]

# 리스트를 데이터프레임으로 변환
df = pd.DataFrame(data_tuples, columns=['Metric', 'Value'])

# 'Value' 컬럼의 문자열을 적절한 타입으로 변환 (숫자 혹은 딕셔너리)
# 문자열이 수치인 경우 float로, 딕셔너리 형태의 문자열인 경우 eval 함수로 실제 딕셔너리 객체로 변환
df['Value'] = df['Value'].apply(lambda x: eval(x) if '{' in x else float(x))

df.head()  # 처음 몇 줄만 표시하여 데이터프레임


ValueError: could not convert string to float: '[0.1369125329239554, 0.029653358697187972, 0.008238131732550294, 0.003247435172234743]'

In [8]:
import ast

# 문자열로 된 리스트
string_list = "[0.11624369729050006, 0.03295186890531648, 0.011771572112776907, 0.004210891852117306]"

# ast.literal_eval()을 사용하여 문자열을 리스트로 변환
converted_list = ast.literal_eval(string_list)

print(converted_list)

[0.11624369729050006, 0.03295186890531648, 0.011771572112776907, 0.004210891852117306]


In [9]:
# 파일 내용을 데이터프레임으로 변환하는 함수 정의
import pandas as pd
import ast
def process_file_to_df(file_path, task):
    # 파일 내용을 한 줄씩 읽어와서 리스트로 변환
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # 각 줄을 '=' 기준으로 나누어 튜플로 변환 후 리스트에 저장
    data_tuples = [tuple(map(str.strip, line.split('='))) for line in lines]

    # 데이터 튜플에서 Metric과 Value 분리
    metrics = [t[0] for t in data_tuples]
    values = [t[1] for t in data_tuples]

    # 'Value' 컬럼의 문자열을 적절한 타입으로 변환 (숫자 혹은 딕셔너리)
    # 문자열이 수치인 경우 float로, 딕셔너리 형태의 문자열인 경우 eval 함수로 실제 딕셔너리 객체로 변환
    try:
        values = [eval(value) if ('{' in value) else float(value) for value in values]
    except:
        values = [eval(value) if ('{' in value) else value for value in values]

        
    # Metric을 컬럼으로 하고 Value를 값으로 하는 데이터프레임 생성
    df = pd.DataFrame(data=values, index=metrics).transpose()
    df = df.drop('sacre_blue_score', axis=1)
    df = df.drop('sacre_blue_score_with_tf', axis=1)
    precisions = ast.literal_eval(df['corpus_bleu_score'].values[0])
    # Adding them as separate columns to the dataframe
    df['BLEU-1'] = precisions[0]
    df['BLEU-2'] = precisions[1]
    df['BLEU-3'] = precisions[2]
    df['BLEU-4'] = precisions[3]
    df = df.drop('corpus_bleu_score', axis=1)
    # Reordering the dataframe to move the BLEU-n columns to the front
    try:
        df['ROUGE_R'], df['ROUGE_P'], df['ROUGE_F']  = df['rouge_scores'].values[0]['rouge-1']['r'], df['rouge_scores'].values[0]['rouge-1']['p'], df['rouge_scores'].values[0]['rouge-1']['f']
        df = df.drop('rouge_scores', axis=1)
    except:
        df['ROUGE_R'], df['ROUGE_P'], df['ROUGE_F']  = 0, 0, 0
        df = df.drop('rouge_scores', axis=1)
        
    precisions = ast.literal_eval(df['corpus_bleu_score_with_tf'].values[0])
    # Adding them as separate columns to the dataframe
    df['BLEU-1 w/tf'] = precisions[0]
    df['BLEU-2 w/tf'] = precisions[1]
    df['BLEU-3 w/tf'] = precisions[2]
    df['BLEU-4 w/tf'] = precisions[3]
    df = df.drop('corpus_bleu_score_with_tf', axis=1)
    
    df['ROUGE_R w/tf'], df['ROUGE_P w/tf'], df['ROUGE_F w/tf']  = df['rouge_scores_with_tf'].values[0]['rouge-1']['r'], df['rouge_scores_with_tf'].values[0]['rouge-1']['p'], df['rouge_scores_with_tf'].values[0]['rouge-1']['f']
    
    df = df.drop('rouge_scores_with_tf', axis=1)
    if task==3:
        df['MODEL'] = file_path[32:-4] # 32 when task3 , 35 when taskNRv2
    elif task==2:
        df['MODEL'] = file_path[35:-4] # 32 when task3 , 35 when taskNRv2
        
    order_columns = ['MODEL','BLEU-1', 'BLEU-2', 'BLEU-3', 'BLEU-4', 'ROUGE_R', 'ROUGE_P', 'ROUGE_F',
                     'wer_scores', 'cer_scores', 'BLEU-1 w/tf', 'BLEU-2 w/tf', 'BLEU-3 w/tf', 'BLEU-4 w/tf', 'ROUGE_R w/tf',
                    'ROUGE_P w/tf', 'ROUGE_F w/tf'
                    ]
    other_columns = [col for col in df.columns if col not in order_columns]
    new_order = order_columns + other_columns
    # Reordered dataframe
    df = df[new_order]

    rename_columns = {
    'wer_scores': 'WER',
    'cer_scores': 'CER',
    'wer_scores_with_tf': 'WER w/tf',
    'cer_scores_with_tf': 'CER w/tf',
    }
    df.rename(columns=rename_columns, inplace=True)
    df.head()
    
    return df



In [31]:

# 파일 경로
BrainTranslator = 'score_results/task1_task2_task3-BrainTranslator-noise_noise.txt'
PegasusTranslator = 'score_results/task1_task2_task3-PegasusTranslator-noise_noise.txt'
T5Translator = 'score_results/task1_task2_task3-T5Translator-noise_noise.txt'

BrainTranslator = process_file_to_df(BrainTranslator, 3)
PegasusTranslator = process_file_to_df(PegasusTranslator, 3)
T5Translator = process_file_to_df(T5Translator, 3)


In [32]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0)

,MODEL,BLEU-1,BLEU-2,BLEU-3,BLEU-4,ROUGE_R,ROUGE_P,ROUGE_F,WER,CER,BLEU-1 w/tf,BLEU-2 w/tf,BLEU-3 w/tf,BLEU-4 w/tf,ROUGE_R w/tf,ROUGE_P w/tf,ROUGE_F w/tf,WER w/tf,CER w/tf
0,BrainTranslator-noise_noise,0.142196,0.030564,0.009322,3.879516e-03,0.116195,0.122851,0.111860,1.1097633815759247,0.8816275728543221,0.396899,0.221722,0.124954,0.071192,0.263159,0.310341,0.283721,0.7808867447737192,0.6018884885177133
0,PegasusTranslator-noise_noise,0.090578,0.026013,0.008379,2.760279e-79,0.000000,0.000000,0.000000,0.9923517288756584,0.8129466097865464,0.391730,0.217043,0.119571,0.061847,0.274149,0.313430,0.291383,0.7810396152965422,0.612199100844777
0,T5Translator-noise_noise,0.163740,0.056892,0.016882,5.876373e-03,0.111934,0.128008,0.107418,1.235820203892493,0.9295996279213984,0.341945,0.183043,0.102142,0.056808,0.183976,0.260426,0.214473,0.8461538461538461,0.687702027053215


In [33]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0).to_csv('task1_task2_task3_noise_noise_results.csv')

In [37]:

# 파일 경로
BrainTranslator = 'score_results/task1_task2_taskNRv2-BrainTranslator-noise_noise.txt'
PegasusTranslator = 'score_results/task1_task2_taskNRv2-PegasusTranslator-noise_noise.txt'
T5Translator = 'score_results/task1_task2_taskNRv2-T5Translator-noise_noise.txt'

BrainTranslator = process_file_to_df(BrainTranslator, 2)
PegasusTranslator = process_file_to_df(PegasusTranslator, 2)
T5Translator = process_file_to_df(T5Translator, 2)


In [38]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0)

,MODEL,BLEU-1,BLEU-2,BLEU-3,BLEU-4,ROUGE_R,ROUGE_P,ROUGE_F,WER,CER,BLEU-1 w/tf,BLEU-2 w/tf,BLEU-3 w/tf,BLEU-4 w/tf,ROUGE_R w/tf,ROUGE_P w/tf,ROUGE_F w/tf,WER w/tf,CER w/tf
0,BrainTranslator-noise_noise,0.117827,0.031821,0.012263,0.005240,0.112172,0.150543,0.120712,1.0111714124612348,0.8097023256907149,0.416100,0.246855,0.143981,0.083958,0.295510,0.357689,0.322683,0.747638849732168,0.5891730224577815
0,PegasusTranslator-noise_noise,0.090764,0.025403,0.009154,0.002158,0.000000,0.000000,0.000000,0.9839696266610419,0.8098172097222385,0.417267,0.238029,0.132934,0.072242,0.306945,0.353145,0.327346,0.7487871757013288,0.5945960205888215
0,T5Translator-noise_noise,0.131937,0.031299,0.009616,0.004220,0.066879,0.050566,0.053054,1.28576496044837,0.9721298673639622,0.395989,0.224964,0.125425,0.069229,0.225837,0.312602,0.260480,0.862580256110106,0.7849722541027249


In [39]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0).to_csv('task1_task2_taskNRv2_noise_noise_results.csv')

In [17]:

# 파일 경로
BrainTranslator = 'score_results/task1_task2_taskNRv2-BrainTranslator-EEG_noise.txt'
PegasusTranslator = 'score_results/task1_task2_taskNRv2-PegasusTranslator-EEG_noise.txt'
T5Translator = 'score_results/task1_task2_taskNRv2-T5Translator-EEG_noise.txt'

BrainTranslator = process_file_to_df(BrainTranslator, 2)
PegasusTranslator = process_file_to_df(PegasusTranslator, 2)
T5Translator = process_file_to_df(T5Translator, 2)


In [18]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0)

,MODEL,BLEU-1,BLEU-2,BLEU-3,BLEU-4,ROUGE_R,ROUGE_P,ROUGE_F,WER,CER,BLEU-1 w/tf,BLEU-2 w/tf,BLEU-3 w/tf,BLEU-4 w/tf,ROUGE_R w/tf,ROUGE_P w/tf,ROUGE_F w/tf,WER w/tf,CER w/tf
0,BrainTranslator-EEG_noise,0.112640,0.031888,0.012781,0.005652,0.110698,0.150776,0.120631,1.0003171694389625,0.798285405379996,0.412190,0.241824,0.138720,0.077750,0.295182,0.363188,0.324231,0.7531012122920778,0.5892905409375624
0,PegasusTranslator-EEG_noise,0.111444,0.027492,0.008782,0.003008,0.000000,0.000000,0.000000,0.9969767278351965,0.8080584835090105,0.410562,0.231687,0.128175,0.068585,0.300769,0.345942,0.320678,0.7551149546509175,0.6022582044577847
0,T5Translator-EEG_noise,0.170200,0.059433,0.020529,0.009482,0.120242,0.124457,0.114964,1.1361072682771096,0.8715228888862596,0.450642,0.270633,0.163214,0.096391,0.269722,0.293565,0.279802,1.0274200986130326,2.058591762603973


In [19]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0).to_csv('task1_task2_taskNRv2_EEG_noise_results.csv')

In [12]:

# 파일 경로
BrainTranslator = 'score_results/task1_task2_taskNRv2-BrainTranslator.txt'
PegasusTranslator = 'score_results/task1_task2_taskNRv2-PegasusTranslator.txt'
T5Translator = 'score_results/task1_task2_taskNRv2-T5Translator.txt'

BrainTranslator = process_file_to_df(BrainTranslator, 2)
PegasusTranslator = process_file_to_df(PegasusTranslator, 2)
T5Translator = process_file_to_df(T5Translator, 2)


In [16]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0)

,MODEL,BLEU-1,BLEU-2,BLEU-3,BLEU-4,ROUGE_R,ROUGE_P,ROUGE_F,WER,CER,BLEU-1 w/tf,BLEU-2 w/tf,BLEU-3 w/tf,BLEU-4 w/tf,ROUGE_R w/tf,ROUGE_P w/tf,ROUGE_F w/tf,WER w/tf,CER w/tf
0,BrainTranslator-EEG_noise,0.112640,0.031888,0.012781,0.005652,0.110698,0.150776,0.120631,1.0003171694389625,0.798285405379996,0.412190,0.241824,0.138720,0.077750,0.295182,0.363188,0.324231,0.7531012122920778,0.5892905409375624
0,PegasusTranslator-EEG_noise,0.111444,0.027492,0.008782,0.003008,0.000000,0.000000,0.000000,0.9969767278351965,0.8080584835090105,0.410562,0.231687,0.128175,0.068585,0.300769,0.345942,0.320678,0.7551149546509175,0.6022582044577847
0,T5Translator-EEG_noise,0.170200,0.059433,0.020529,0.009482,0.120242,0.124457,0.114964,1.1361072682771096,0.8715228888862596,0.450642,0.270633,0.163214,0.096391,0.269722,0.293565,0.279802,1.0274200986130326,2.058591762603973


In [20]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0)#.to_csv('task1_task2_taskNRv2_results.csv')

,MODEL,BLEU-1,BLEU-2,BLEU-3,BLEU-4,ROUGE_R,ROUGE_P,ROUGE_F,WER,CER,BLEU-1 w/tf,BLEU-2 w/tf,BLEU-3 w/tf,BLEU-4 w/tf,ROUGE_R w/tf,ROUGE_P w/tf,ROUGE_F w/tf,WER w/tf,CER w/tf
0,BrainTranslator-EEG_noise,0.112640,0.031888,0.012781,0.005652,0.110698,0.150776,0.120631,1.0003171694389625,0.798285405379996,0.412190,0.241824,0.138720,0.077750,0.295182,0.363188,0.324231,0.7531012122920778,0.5892905409375624
0,PegasusTranslator-EEG_noise,0.111444,0.027492,0.008782,0.003008,0.000000,0.000000,0.000000,0.9969767278351965,0.8080584835090105,0.410562,0.231687,0.128175,0.068585,0.300769,0.345942,0.320678,0.7551149546509175,0.6022582044577847
0,T5Translator-EEG_noise,0.170200,0.059433,0.020529,0.009482,0.120242,0.124457,0.114964,1.1361072682771096,0.8715228888862596,0.450642,0.270633,0.163214,0.096391,0.269722,0.293565,0.279802,1.0274200986130326,2.058591762603973


In [57]:

# 파일 경로
BrainTranslator = 'score_results/task1_task2_taskNRv2-BrainTranslator-noise_EEG.txt'
PegasusTranslator = 'score_results/task1_task2_taskNRv2-PegasusTranslator-noise_EEG.txt'
T5Translator = 'score_results/task1_task2_taskNRv2-T5Translator-noise_EEG.txt'

BrainTranslator = process_file_to_df(BrainTranslator, 2)
PegasusTranslator = process_file_to_df(PegasusTranslator, 2)
T5Translator = process_file_to_df(T5Translator, 2)


In [21]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0)#.to_csv('task1_task2_taskNRv2_noise_EEG_results.csv')

,MODEL,BLEU-1,BLEU-2,BLEU-3,BLEU-4,ROUGE_R,ROUGE_P,ROUGE_F,WER,CER,BLEU-1 w/tf,BLEU-2 w/tf,BLEU-3 w/tf,BLEU-4 w/tf,ROUGE_R w/tf,ROUGE_P w/tf,ROUGE_F w/tf,WER w/tf,CER w/tf
0,BrainTranslator-EEG_noise,0.112640,0.031888,0.012781,0.005652,0.110698,0.150776,0.120631,1.0003171694389625,0.798285405379996,0.412190,0.241824,0.138720,0.077750,0.295182,0.363188,0.324231,0.7531012122920778,0.5892905409375624
0,PegasusTranslator-EEG_noise,0.111444,0.027492,0.008782,0.003008,0.000000,0.000000,0.000000,0.9969767278351965,0.8080584835090105,0.410562,0.231687,0.128175,0.068585,0.300769,0.345942,0.320678,0.7551149546509175,0.6022582044577847
0,T5Translator-EEG_noise,0.170200,0.059433,0.020529,0.009482,0.120242,0.124457,0.114964,1.1361072682771096,0.8715228888862596,0.450642,0.270633,0.163214,0.096391,0.269722,0.293565,0.279802,1.0274200986130326,2.058591762603973


In [68]:
torch.randn(1,3)

tensor([[-0.0783,  0.1029, -0.0224]])

In [70]:
torch.randn(1,3)

tensor([[ 0.6236, -0.9636,  1.4547]])

In [12]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0).to_csv('task1_task2_taskNRv2_EEG_noise_results.csv')

In [43]:

# 파일 경로
BrainTranslator = 'score_results/task1_task2_task3-BrainTranslator-EEG_noise.txt'
PegasusTranslator = 'score_results/task1_task2_task3-PegasusTranslator-EEG_noise.txt'
T5Translator = 'score_results/task1_task2_task3-T5Translator-EEG_noise.txt'

BrainTranslator = process_file_to_df(BrainTranslator, 3)
PegasusTranslator = process_file_to_df(PegasusTranslator, 3)
T5Translator = process_file_to_df(T5Translator, 3)


In [45]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0).to_csv('task1_task2_task3_EEG_noise_results.csv')

In [15]:

# 파일 경로
BrainTranslator = 'score_results/task1_task2_task3-BrainTranslator.txt'
PegasusTranslator = 'score_results/task1_task2_task3-PegasusTranslator.txt'
T5Translator = 'score_results/task1_task2_task3-T5Translator.txt'

BrainTranslator = process_file_to_df(BrainTranslator, 3)
PegasusTranslator = process_file_to_df(PegasusTranslator, 3)
T5Translator = process_file_to_df(T5Translator, 3)


In [46]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0)#.to_csv('task1_task2_task3_results.csv')

,MODEL,BLEU-1,BLEU-2,BLEU-3,BLEU-4,ROUGE_R,ROUGE_P,ROUGE_F,WER,CER,BLEU-1 w/tf,BLEU-2 w/tf,BLEU-3 w/tf,BLEU-4 w/tf,ROUGE_R w/tf,ROUGE_P w/tf,ROUGE_F w/tf,WER w/tf,CER w/tf
0,BrainTranslator-EEG_noise,0.138750,0.030893,0.007741,2.464770e-03,0.122318,0.135972,0.121439,1.08306914771422,0.8548675829959204,0.393433,0.221301,0.125187,0.072850,0.264403,0.314283,0.286146,0.7806570181484034,0.6020344660679026
0,PegasusTranslator-EEG_noise,0.085760,0.024848,0.007782,2.607656e-79,0.000000,0.000000,0.000000,0.998855049232883,0.8127627961123408,0.382988,0.210943,0.115708,0.061212,0.268420,0.306464,0.285083,0.7856194183650103,0.6161204592278294
0,T5Translator-EEG_noise,0.102951,0.017306,0.003523,1.217001e-03,0.031462,0.028157,0.027059,1.2862835959221501,1.0555404829270183,0.426760,0.242983,0.141400,0.080724,0.213588,0.255947,0.231493,0.8549119555143652,2.1924576566799736


In [38]:

# 파일 경로
BrainTranslator = 'score_results/task1_task2_taskNRv2-BrainTranslator-noise_EEG.txt'
PegasusTranslator = 'score_results/task1_task2_taskNRv2-PegasusTranslator-noise_EEG.txt'
T5Translator = 'score_results/task1_task2_taskNRv2-T5Translator-noise_EEG.txt'

BrainTranslator = process_file_to_df(BrainTranslator, 2)
PegasusTranslator = process_file_to_df(PegasusTranslator, 2)
T5Translator = process_file_to_df(T5Translator, 2)


In [39]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0).to_csv('task1_task2_taskNRv2_noise_EEG_results.csv')

In [42]:

# 파일 경로
BrainTranslator = 'score_results/task1_task2_task3-BrainTranslator-noise_EEG.txt'
PegasusTranslator = 'score_results/task1_task2_task3-PegasusTranslator-noise_EEG.txt'
T5Translator = 'score_results/task1_task2_task3-T5Translator-noise_EEG.txt'

BrainTranslator = process_file_to_df(BrainTranslator, 3)
PegasusTranslator = process_file_to_df(PegasusTranslator, 3)
T5Translator = process_file_to_df(T5Translator, 3)


In [43]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0).to_csv('task1_task2_task3_noise_EEG_results.csv')

In [134]:

# 파일 경로
BrainTranslator = 'score_results/task1_task2_task3-BrainTranslator.txt'
PegasusTranslator = 'score_results/task1_task2_task3-PegasusTranslator.txt'
T5Translator = 'score_results/task1_task2_task3-T5Translator.txt'

BrainTranslator = process_file_to_df(BrainTranslator)
PegasusTranslator = process_file_to_df(PegasusTranslator)
T5Translator = process_file_to_df(T5Translator)


In [145]:

# 파일 경로
BrainTranslator = 'score_results/task1_task2_taskNRv2-BrainTranslator.txt'
PegasusTranslator = 'score_results/task1_task2_taskNRv2-PegasusTranslator.txt'
T5Translator = 'score_results/task1_task2_taskNRv2-T5Translator.txt'

BrainTranslator = process_file_to_df(BrainTranslator)
PegasusTranslator = process_file_to_df(PegasusTranslator)
T5Translator = process_file_to_df(T5Translator)


In [146]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0)

,MODEL,BLEU-1,BLEU-2,BLEU-3,BLEU-4,ROUGE_R,ROUGE_P,ROUGE_F,WER,CER,BLEU-1 w/tf,BLEU-2 w/tf,BLEU-3 w/tf,BLEU-4 w/tf,ROUGE_R w/tf,ROUGE_P w/tf,ROUGE_F w/tf,WER w/tf,CER w/tf
0,BrainTranslator,0.117954,0.033920,0.012112,0.004071,0.100355,0.129007,0.106243,1.0437399494601425,0.8119962814139846,0.380889,0.209755,0.113932,0.059554,0.258295,0.319586,0.284029,0.7903514817367333,0.6071590463824458
0,PegasusTranslator,0.108411,0.028725,0.007907,0.002117,0.000000,0.000000,0.000000,1.0256926952141059,0.8200157773403693,0.387217,0.214246,0.117665,0.060956,0.266898,0.306920,0.284386,0.7826883444011907,0.6158753743288886
0,T5Translator,0.166737,0.057687,0.018347,0.006993,0.109538,0.100305,0.097729,1.2196478220574607,0.9238479128715942,0.424061,0.241839,0.142301,0.080540,0.226514,0.245723,0.234426,1.1361909175162188,2.0174334328126817


In [ ]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0).to_csv('task1_task2_task3_results.csv')

In [136]:
pd.concat([BrainTranslator, PegasusTranslator, T5Translator], axis=0).to_csv('task1_task2_task3_results.csv')

In [90]:
BrainTranslator = 'score_results/task1_task2_task3-BrainTranslator.txt'
PegasusTranslator = 'score_results/task1_task2_task3-PegasusTranslator.txt'
# T5Translator = 'score_results/task1_task2_task3-T5Translator.txt'

BrainTranslator = process_file_to_df(BrainTranslator)
PegasusTranslator = process_file_to_df(PegasusTranslator)
# T5Translator = process_file_to_df(T5Translator)


In [92]:
pd.concat([BrainTranslator, PegasusTranslator], axis=0)

,MODEL,BLEU-1,BLEU-2,BLEU-3,BLEU-4,ROUGE_R,ROUGE_P,ROUGE_F,WER,CER,BLEU-1 w/tf,BLEU-2 w/tf,BLEU-3 w/tf,BLEU-4 w/tf,ROUGE_R w/tf,ROUGE_P w/tf,ROUGE_F w/tf,WER w/tf,CER w/tf
0,BrainTranslator,16.748466,1.286210,0.169866,0.01244,0.104436,0.106459,0.098566,1.239283,0.921971,36.371650,9.540844,2.477507,0.884624,0.243653,0.299431,0.267439,0.795314,0.614827
0,PegasusTranslator,13.149917,0.495771,0.041284,0.01100,0.000000,0.000000,0.000000,1.1616212502862377,0.894406703071986,35.300513,8.701849,2.136130,0.518906,0.244516,0.280539,0.260232,0.8116326997939088,0.6334295802155215


In [94]:
2e-5

2e-05

In [96]:
0.00002

2e-05